# VAE

In [ ]:
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from os import listdir

from PIL import Image

In [ ]:
def load_real_samples(scale=False):
    # We load 20,000 samples only to avoid memory issues, you can  change this value
    X = np.load("cat_img.npy")
    # Scale samples in range [-127, 127]
    if scale:
        X = (X - 127.5) * 2
    return X / 255.

In [ ]:
dataset = load_real_samples()

In [ ]:
def grid_plot(images, epoch='', name='', n=3, save=False, scale=False):
    if scale:
        images = (images + 1) / 2.0
    for index in range(n * n):
        plt.subplot(n, n, 1 + index)
        plt.axis('off')
        plt.imshow(images[index])
    fig = plt.gcf()
    fig.suptitle(name + '  '+ str(epoch), fontsize=14)
    if save:
        filename = '/results/'+name+'.png' 
        plt.savefig(filename)
        plt.close()
    plt.show()


grid_plot(dataset[np.random.randint(0, 1000, 9)], name='Fliqr dataset (64x64x3)', n=3)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv2DTranspose, Reshape

def build_conv_net(in_shape, out_shape, n_downsampling_layers=4, filters=128, out_activation='sigmoid'):
    """
    Build a basic convolutional network
    """
    model = tf.keras.Sequential()
    default_args=dict(kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')

    model.add(Conv2D(input_shape=in_shape, **default_args, filters=filters))

    for _ in range(n_downsampling_layers):
        model.add(Conv2D(**default_args, filters=filters))

    model.add(Flatten())
    model.add(Dense(out_shape, activation=out_activation) )
    model.summary()
    return model


def build_deconv_net(latent_dim, n_upsampling_layers=4, filters=128, activation_out='sigmoid'):
    """
    Build a deconvolutional network for decoding/upscaling latent vectors

    When building the deconvolutional architecture, usually it is best to use the same layer sizes that 
    were used in the downsampling network and the Conv2DTranspose layers are used instead of Conv2D layers. 
    Using identical layers and hyperparameters ensures that the dimensionality of our output matches the
    shape of our input images. 
    """

    model = tf.keras.Sequential()
    model.add(Dense(4 * 4 * 64, input_dim=latent_dim)) 
    model.add(Reshape((4, 4, 64))) # This matches the output size of the downsampling architecture
    default_args=dict(kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')
    
    for i in range(n_upsampling_layers):
        model.add(Conv2DTranspose(**default_args, filters=filters))

    # This last convolutional layer converts back to 3 channel RGB image
    model.add(Conv2D(filters=3, kernel_size=(3,3), activation=activation_out, padding='same'))
    model.summary()
    return model

In [ ]:
class Sampling(tf.keras.layers.Layer):
    """
    Custom layer for the variational autoencoder
    It takes two vectors as input - one for means and other for variances of the latent variables described by a multimodal gaussian
    Its output is a latent vector randomly sampled from this distribution
    """
    def call(self, inputs):
        z_mean, z_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_var) * epsilon

def build_vae(data_shape, latent_dim, filters=128):

    # Building the encoder - starts with a simple downsampling convolutional network  
    encoder = build_conv_net(data_shape, latent_dim*2, filters=filters)
    
    # Adding special sampling layer that uses the reparametrization trick 
    z_mean = Dense(latent_dim)(encoder.output)
    z_var = Dense(latent_dim)(encoder.output)
    z = Sampling()([z_mean, z_var])
    
    # Connecting the two encoder parts
    encoder = tf.keras.Model(inputs=encoder.input, outputs=z)

    # Defining the decoder which is a regular upsampling deconvolutional network
    decoder = build_deconv_net(latent_dim, activation_out='sigmoid', filters=filters)
    vae = tf.keras.Model(inputs=encoder.input, outputs=decoder(z))
    
    # Adding the special loss term
    kl_loss = -0.5 * tf.reduce_sum(z_var - tf.square(z_mean) - tf.exp(z_var) + 1)
    vae.add_loss(kl_loss/tf.cast(tf.keras.backend.prod(data_shape), tf.float32))

    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy')

    return encoder, decoder, vae

In [ ]:
latent_dim = 32
encoder, decoder, vae = build_vae(dataset.shape[1:], latent_dim, filters=128)
loss = []
# Generate random vectors that we will use to sample our latent space
for epoch in range(50):
    latent_vectors = np.random.randn(9, latent_dim)
    history=vae.fit(x=dataset, y=dataset, epochs=1, batch_size=8)
    loss.append(history)
    images = decoder(latent_vectors)
    grid_plot(images, epoch, name='VAE generated images (randomly sampled from the latent space)', n=3, save=False)

In [ ]:
latent_dim = 32
noise = np.random.randn(1, latent_dim)

In [ ]:
vec_1 = np.random.randn(1, latent_dim)
vec_2 = np.random.randn(1, latent_dim)

def interpolate_points(p1, p2, n_steps=10):
	# interpolate ratios between the points
  ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
  print(latent_dim)
  vectors = np.zeros((n_steps,latent_dim))
  for i in range(len(ratios)):
    vectors[i] = (1.0 - ratios[i]) * p1 + ratios[i] * p2
  return vectors
 
def plot_generated(examples, n, save = True):
	# plot images
	for i in range(n):
		plt.subplot(1, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :])

def plot_generated(examples, n):
  for i in range(n):
    plt.subplot(1, n, 1 + i)
    plt.axis('off')
    plt.imshow(examples[i, :, :])

  plt.gcf().set_size_inches(45, 15)
  plt.savefig('results/VAE_lat.png')
  

interpolated = interpolate_points(vec_1, vec_2)
print(np.shape(interpolated))

images = decoder.predict(interpolated)
#images = (images + 1) / 2.0
plot_generated(images, len(interpolated))

# GAN

In [ ]:
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from os import listdir

from PIL import Image

from tensorflow import keras

In [ ]:
dataset = np.load("cat_img.npy")

In [ ]:
from matplotlib import pyplot as plt


img = plt.imshow(dataset[0])
img.set_cmap('hot')
plt.axis('off')


In [ ]:
def load_real_samples(dataset,scale=False):
    # We load 20,000 samples only to avoid memory issues, you can  change this value
    X = dataset
    # Scale samples in range [-127, 127]
    if scale:
        X = (X - 127.5) * 2
    return X / 255.

In [ ]:
def grid_plot(images, epoch='', name='', n=3, save=False, scale=False):
    if scale:
        images = (images + 1) / 2.0
    
    for index in range(n * n):

        plt.subplot(n, n, 1 + index)
        plt.axis('off')
        plt.imshow(images[index])
    fig = plt.gcf()
    fig.suptitle(name + '  ', fontsize=14)
    if save:
        filename = '/results/'+ name +'.png'
        plt.savefig(filename)
        plt.close()
    plt.show()

grid_plot(dataset[np.random.randint(0, 1000, 9)], name='Fliqr dataset (64x64x3)', n=1)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv2DTranspose, Reshape, BatchNormalization, LeakyReLU, GaussianNoise

def build_conv_net(in_shape, out_shape, n_downsampling_layers=4, filters=128, out_activation='sigmoid'):
    """
    Build a basic convolutional network
    """
    model = tf.keras.Sequential()
    default_args=dict(kernel_size=(3,3), strides=(2,2), padding='same')

    model.add(Conv2D(input_shape=in_shape, kernel_size=(3,3), strides=(2,2), padding='same', filters=filters))
    model.add(LeakyReLU())
    

    for _ in range(n_downsampling_layers):
        model.add(Conv2D(**default_args, filters=filters))
        model.add(LeakyReLU())
        model.add(GaussianNoise(0.01))
        
    model.add(Flatten())
    model.add(Dense(out_shape, activation='linear') )
    model.summary()
    return model


def build_deconv_net(latent_dim, n_upsampling_layers=4, filters=128, activation_out='sigmoid'):
    """
    Build a deconvolutional network for decoding/upscaling latent vectors

    When building the deconvolutional architecture, usually it is best to use the same layer sizes that 
    were used in the downsampling network and the Conv2DTranspose layers are used instead of Conv2D layers. 
    Using identical layers and hyperparameters ensures that the dimensionality of our output matches the
    shape of our input images. 
    """

    model = tf.keras.Sequential()
    model.add(Dense(4 * 4 * 64, input_dim=latent_dim)) 
    model.add(Reshape((4, 4, 64))) # This matches the output size of the downsampling architecture
    default_args=dict(kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')

    
    for i in range(n_upsampling_layers):
        model.add(Conv2DTranspose(**default_args, filters=filters))
       
    # This last convolutional layer converts back to 3 channel RGB image
    model.add(Conv2D(filters=3, kernel_size=(3,3), activation=activation_out, padding='same'))
    model.summary()
    return model

In [ ]:
def build_gan(data_shape, latent_dim, filters=128, lr=0.0002, beta_1=0.5):
    optimizer = tf.optimizers.Adam(learning_rate=lr, beta_1=beta_1)

    # Usually thew GAN generator has tanh activation function in the output layer
    generator = build_deconv_net(latent_dim, activation_out='tanh', filters=filters)
    
    # Build and compile the discriminator
    discriminator = build_conv_net(in_shape=data_shape, out_shape=1, filters=filters) # Single output for binary classification
    discriminator.compile(loss='mse', optimizer=optimizer, metrics = ['accuracy'])
    
    # End-to-end GAN model for training the generator
    discriminator.trainable = False
    true_fake_prediction = discriminator(generator.output)
    GAN = tf.keras.Model(inputs=generator.input, outputs=true_fake_prediction)
    GAN = tf.keras.models.Sequential([generator, discriminator])
    GAN.compile(loss='mse', optimizer=optimizer)
    
    return discriminator, generator, GAN

In [ ]:
def run_generator(generator, n_samples=100):
    """
    Run the generator model and generate n samples of synthetic images using random latent vectors
    """
    latent_dim = generator.layers[0].input_shape[-1]
    generator_input = np.random.randn(n_samples, latent_dim)

    return generator.predict(generator_input)
    

def get_batch(generator, dataset, batch_size=32):
    """
    Gets a single batch of samples (X) and labels (y) for the training the discriminator.
    One half from the real dataset (labeled as 1s), the other created by the generator model (labeled as 0s).
    """
    batch_size //= 2 # Split evenly among fake and real samples

    fake_data = run_generator(generator, n_samples=batch_size)
    real_data = dataset[np.random.randint(0, dataset.shape[0], batch_size)]

    X = np.concatenate([fake_data, real_data], axis=0)
    y = np.concatenate([np.zeros([batch_size, 1]), np.ones([batch_size, 1])], axis=0)

    return X, y


def train_gan(generator, discriminator, gan, dataset, latent_dim, n_epochs=20, batch_size=32):

    batches_per_epoch = int(dataset.shape[0] / batch_size / 2)

    acc = np.zeros(shape=(n_epochs,batches_per_epoch))

    for epoch in range(n_epochs):
        i = 0
        for batch in tqdm(range(batches_per_epoch)):
            
            # 1) Train discriminator both on real and synthesized images
            X, y = get_batch(generator, dataset, batch_size=batch_size)
            discriminator_loss, discriminator_acc = discriminator.train_on_batch(X, y)

            acc[epoch][i] = discriminator_acc

            # 2) Train generator (note that now the label of synthetic images is reversed to 1)
            X_gan = np.random.randn(batch_size, latent_dim)
            y_gan = np.ones([batch_size, 1])
            generator_loss = gan.train_on_batch(X_gan, y_gan)

            i = i+1

        noise = np.random.randn(16, latent_dim)
        images = generator.predict(noise)
        grid_plot(images, epoch, name='GAN generated images', n=3, save=False, scale=True)
   
    return acc

In [ ]:
latent_dim = 256
discriminator, generator, gan = build_gan(dataset.shape[1:], latent_dim, filters=128)
dataset_scaled = load_real_samples(dataset,scale=True)

acc = train_gan(generator, discriminator, gan, dataset_scaled, latent_dim, n_epochs=50)

In [ ]:
latent_dim = 256
noise = np.random.randn(1, latent_dim)

In [ ]:
images = generator.predict(noise)
grid_plot(images, 0, name='GAN generated images', n=1, save=False, scale=True)

In [ ]:
noise = np.random.randn(1, 256)
latent_dim=256

In [ ]:
vec_1 = np.random.randn(1, latent_dim)
vec_2 = np.random.randn(1, latent_dim)

In [ ]:
def interpolate_points(p1, p2, n_steps=10):
	# interpolate ratios between the points
  ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
  vectors = np.zeros((n_steps,256))
  for i in range(len(ratios)):
    vectors[i] = (1.0 - ratios[i]) * p1 + ratios[i] * p2
  return vectors
 
def plot_generated(examples, n, save = True):
	# plot images
	for i in range(n):
		plt.subplot(1, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :])

def plot_generated(examples, n):
  for i in range(n):
    plt.subplot(1, n, 1 + i)
    plt.axis('off')
    plt.imshow(examples[i, :, :])

  plt.gcf().set_size_inches(45, 15)
  plt.savefig('results/GAN_lat.png')
  #plt.close()
  

interpolated = interpolate_points(vec_1, vec_2)
print(np.shape(interpolated))

images = generator.predict(interpolated)
images = (images + 1) / 2.0
plot_generated(images, len(interpolated))